### 1. Mineria de Datos sobre Redes Sociales y Sitios Web

#### 1.1. Tomando datos de la red social Twitter, al menos 200 tweets, realice un script que recolecte, procese y vizualice los datos con las siguientes restricciones:

##### a)Los tres trends más RT del momento.

In [1]:
#Cargamos los paquetes necesarios para entrar a la API de Twitter
import twitter
import json
from prettytable import PrettyTable
from collections import Counter
import pandas as pd

keys = pd.read_json('keys.json')['KEYS']
CONSUMER_KEY = keys['CONSUMER_KEY']
CONSUMER_SECRET = keys['CONSUMER_SECRET']
OAUTH_TOKEN = keys['OAUTH_TOKEN']
OAUTH_TOKEN_SECRET = keys['OAUTH_TOKEN_SECRET']

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

#### Seteamos los WOEID para obtener los resultados 

In [2]:
SPAIN_WOE_ID = 23424950
spain_trends = twitter_api.trends.place(_id=SPAIN_WOE_ID)
hashtags = spain_trends[0]['trends']
trends = map(lambda i: hashtags[i]['name'], range(len(hashtags)))[:3]
pt = PrettyTable()
pt.add_column('Spain Trends', trends)
print pt

+--------------+
| Spain Trends |
+--------------+
|  #TopChef2   |
|    Chile     |
|  #LlegaByb   |
+--------------+


#### b )   Listar nombres de usuarios que publicaron con los hashtag de los trends del punto anterior.

In [3]:
# Obtenemos los tweets
CANT = 10
in_spain = set(trends)

tweets = []
for i in range(len(in_spain)):
    results = twitter_api.search.tweets(q=in_spain.pop() , count=CANT)
    tweets += results['statuses']

# Obtenemos los nombres de los users
screen_name = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
screen_name = list(set(screen_name)) #sin repetidos
pt = PrettyTable()
pt.add_column('Users', screen_name )

#mostramos los resultados en una tabla
print pt

+-----------------+
|      Users      |
+-----------------+
|     madgjmi     |
|     mesa1996    |
|   aries13music  |
|  SalarioVirtual |
|    mistak3btr   |
|    mjanemann    |
|  ElObjetivista  |
|    RubiMarinL   |
|   MadaMorenita  |
|     QuiquiR     |
|     ttesbot     |
|     MasiolE     |
| aureliosanchez6 |
| marcosruffo3231 |
|  gumerealmadrid |
| ConyMiniBadGirl |
|  blanca_maybri  |
|     ReVale24    |
|     ducksel     |
| GalitobandaGalo |
|   Valriberas97  |
|  el_gran_rodri  |
|  laiacasanovas  |
+-----------------+


#### c) Cuales son las cinco palabras más utilizadas en los tweets del primer item.

In [4]:
text = map( lambda i: tweets[i]['text'], range(len(tweets)))
words = [ word for twt in text for word in twt.split() ]
c = Counter(words)
#print c.most_common()

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:5])
print pt

+-----------+-------+
|   Words   | Count |
+-----------+-------+
|     RT    |   16  |
|     de    |   13  |
| #TopChef2 |   10  |
|     y     |   10  |
|   Chile   |   8   |
+-----------+-------+


#### d) Listar los primeros 10 usuarios con mayor cantidad de seguidores

In [5]:
lista_users_followers = map( lambda i: (tweets[i]['user']['screen_name'],tweets[i]['user']['followers_count']), range(len(tweets)))

lista_users_followers = sorted(lista_users_followers, key=lambda x: x[1], reverse=True)[:10]
users = map((lambda x: x[0]), lista_users_followers)
followers = map((lambda x: x[1]), lista_users_followers)


pt = PrettyTable()
pt.add_column('screen_name', users)
pt.add_column('followers_count', followers)
print pt

+-----------------+-----------------+
|   screen_name   | followers_count |
+-----------------+-----------------+
|  gumerealmadrid |      26629      |
|     ttesbot     |       9929      |
|   aries13music  |       3015      |
|   aries13music  |       3015      |
|    RubiMarinL   |       1113      |
|     ReVale24    |       993       |
|  ElObjetivista  |       782       |
|     MasiolE     |       769       |
|   Valriberas97  |       713       |
| aureliosanchez6 |       676       |
+-----------------+-----------------+


#### e) Listar la ubicación (o en su defecto time-zone ) del tweet.

In [6]:
users = map( lambda i: tweets[i]['user']['screen_name'], range(len(tweets)))
location_or_tz = map( lambda i: tweets[i]['user']['location'] or tweets[i]['user']['time_zone'], range(len(tweets)))

pt = PrettyTable()
pt.add_column('User', users)
pt.add_column('Location or TZ', location_or_tz)
print pt

+-----------------+--------------------------------+
|       User      |         Location or TZ         |
+-----------------+--------------------------------+
|  SalarioVirtual |              None              |
|     QuiquiR     |              None              |
|   Valriberas97  |        Sevilla y Ceuta         |
|   MadaMorenita  |          Mancha real           |
|     MasiolE     |       En mi espacio...         |
|  ElObjetivista  |             Spain              |
|     ttesbot     |             Madrid             |
|    mjanemann    |              None              |
|    mistak3btr   |           Argentina            |
|     ReVale24    |   Central Time (US & Canada)   |
| aureliosanchez6 |        Ocaña  colombia         |
|     ducksel     |   Christchurch, New Zealand    |
| GalitobandaGalo |              None              |
| marcosruffo3231 |             Bogota             |
| ConyMiniBadGirl |        Que te importa❤️        |
|  gumerealmadrid |        Isla Margarita     

#### f) Listar los cinco tweets más populares.


In [7]:
texto_y_rt = map( lambda i: (tweets[i]['text'], tweets[i]['retweet_count'], tweets[i]['user']['id']), range(len(tweets)))

texto_y_rt = sorted(texto_y_rt, key=lambda x: x[1], reverse=True)[:5]
popular_tw = texto_y_rt[0]
texto = map((lambda x: x[0]), texto_y_rt)
rts = map((lambda x: x[1]), texto_y_rt)


pt = PrettyTable()
pt.add_column('Tweet', texto)
pt.max_width['Tweet'] = 80
pt.add_column('Retweets', rts)
pt.align = 'l'
print pt

+----------------------------------------------------------------------------------+----------+
| Tweet                                                                            | Retweets |
+----------------------------------------------------------------------------------+----------+
| RT @teleaudiencias: #LlegaBYB: ¿Te ha gustado el capítulo? ¿Tenías ganas de su   | 217      |
| regreso?                                                                         |          |
| RT: SI                                                                           |          |
| FAV: NO http://t.co/rJg8uoFPyA                                                   |          |
| RT @NTN24: #EnVivo El Centro Sismológico de la Universidad de Chile eleva a 8.4  | 116      |
| la magnitud del terremoto http://t.co/XEEr95y03t http://t.c…                     |          |
| RT @Sr_Rembrandt: Ir a un BurgerKing y preguntarle al chaval "Qué has querido    | 37       |
| transmitir poniendo pepinillo en el Wh

#### g) Listar a los seguidores del autor del tweet más popular

In [8]:
import tweepy
auth_tweepy = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth_tweepy.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth_tweepy)
user = api.get_user(popular_tw[2])
title = "%s's followers:" % (user.screen_name)
followers = map((lambda f: f.screen_name), user.followers(count=100))

pt = PrettyTable()
pt.add_column(title, followers)
print pt

+---------------------------+
| MadaMorenita's followers: |
+---------------------------+
|        MeteoAndujar       |
|       Gabrielbarrosm      |
|        arieldpoint        |
|       guccipaloma988      |
|      FunKittyGiggles      |
|       arianaloveya3       |
|      Yasminita_17594      |
|       OfficialMartin      |
|         coselcomsa        |
|       reflexionesrc       |
|        susanarj_74        |
|          Simo_RL          |
|         mrsemueve         |
|        3MariasBaeza       |
|          AdassaER         |
|          prices_k         |
|       Xristinaki_Arg      |
|        ortiztere69        |
|         lorse1980         |
|       MamenGallilla       |
|      MariMorillas_96      |
|        macarenacobo       |
|         framdavila        |
|       upimanchareal       |
|       FrasesOdiosas       |
|        espaguetiis        |
|        atreveteYAmr       |
|          kiskofmp         |
|       amaliabadajoz       |
|       ronnieonslow12      |
|         

### 2. Tomando datos de la red Social Google+, recolecte, procese y vizualice los datos con las siguientes restricciones:

#### a) Obtener las tres últimas actividades de X usuario.

In [9]:
import apiclient.discovery
import httplib2

#Accedemos a la api de google+ con la clave correspondiente
API_KEY = keys['G+'] 
service = apiclient.discovery.build('plus', 'v1', 
                                    http=httplib2.Http(), 
                                    developerKey=API_KEY)

people_feed = service.people().search(query='Linus Torvalds').execute()
#print json.dumps(people_feed['items'], indent=1)

#El usuario elegido en este caso es: Linus Torvalds
id_linus = people_feed['items'][0]['id']
activity_feed = service.activities().list(
  userId= id_linus,
  collection='public',
  maxResults='3'
).execute()

print json.dumps(activity_feed, indent=1)

{
 "nextPageToken": "Cg0Qhefojen8xwIgACgBEhQIABDA1pvNlejHAhjAyITS8pXHAhgCIAMo7uuMpf73se-WAQ", 
 "kind": "plus#activityFeed", 
 "title": "Google+ List of Activities for Collection PUBLIC", 
 "items": [
  {
   "kind": "plus#activity", 
   "provider": {
    "title": "Google+"
   }, 
   "title": "Today's poll is in honor of the homeopathy conference attendees tripping balls in Germany. Apparently...", 
   "url": "https://plus.google.com/+LinusTorvalds/posts/gRJpZhuQM6c", 
   "object": {
    "resharers": {
     "totalItems": 224, 
     "selfLink": "https://www.googleapis.com/plus/v1/activities/z13qzlf50uemjjpjf04cj1yx1py4xrsb1ps0k/people/resharers"
    }, 
    "url": "https://plus.google.com/+LinusTorvalds/posts/gRJpZhuQM6c", 
    "actor": {
     "verification": {
      "adHocVerified": "PASSED"
     }
    }, 
    "content": "Today&#39;s poll is in honor of the homeopathy conference attendees tripping balls in Germany. Apparently they weren&#39;t very <b>good</b> at what they do, and didn&#

#### b) Analizar el texto de una de estas actividades. (Palabras utilizadas, cantidad de veces, etc)

In [10]:
feeds_content = []
feeds = activity_feed['items'] 
for i in range(len(feeds)):
    feeds_content.append(feeds[i]['object']['content'])
print(feeds_content)

[u'Today&#39;s poll is in honor of the homeopathy conference attendees tripping balls in Germany. Apparently they weren&#39;t very <b>good</b> at what they do, and didn&#39;t dilute their drugs enough.<br /><br />So in honor of these people finally having drugs that did something, please fill in the blank:<br /><br />&quot;I believe in homeopathy, because I&#39;m ..&quot;\ufeff', u'Poor Samsung.<br /><br />Even when they make a watch that is clearly better looking than the Apple thing, and get good reviews for how their bezel works well and is really functional and intuitive, they can&#39;t seem to catch a break.<br /><br />Most of the reviews I&#39;ve seen tippy-toe around the &quot;better looking than Apple&quot;. I think my favorite euphemism so far is that the Gear S2 &quot;compares favorably&quot; with the Apple offerings.\ufeff', u'This one is possibly titled<br /><br />   &quot;Dang, my laptop is slow to compile&quot;<br /><br />but that&#39;s maybe a bit unfair - I got that lap

In [11]:
words = [ word for twt in feeds_content for word in twt.split() ]
c = Counter(words)

pt = PrettyTable(field_names=['Words', 'Count'])
map(lambda r: pt.add_row(r), c.most_common()[:15])
print pt

+----------+-------+
|  Words   | Count |
+----------+-------+
|   the    |   8   |
|  /><br   |   6   |
|    is    |   6   |
|   and    |   5   |
|    in    |   5   |
|   they   |   4   |
|   that   |   4   |
|    of    |   3   |
| because  |   3   |
|    a     |   3   |
|  laptop  |   2   |
|    to    |   2   |
| looking  |   2   |
| it&#39;s |   2   |
|  their   |   2   |
+----------+-------+


#### c) Visualizar el avatar de tres individuos a seleccionar.

In [1]:
from IPython.display import Image, display

def get_user_avatar(username,width, height):
    people_feed = service.people().search(query=username).execute()
    if len(people_feed['items']) > 0:    
        url = (people_feed['items'][0]['image']['url']).split('?')[0] 
    else:
        url = 'http://placehold.it/{}x{}'.format(width, height)
    return Image(url = url, width = width, height = height )

users = ['Linus Torvalds', 'Guido Van Rossum', 'Larry Page' ]
imgs = map((lambda user: get_user_avatar(user, 150, 150)), users)
_ = map((lambda img: display(img)), imgs)

NameError: global name 'service' is not defined

### 3. Sacando información de Microformatos en la Web

In [48]:
import feedparser

#### a) Obtener y visualizar coordenadas geográficas.

#### b) Listar calendario de eventos.

Obtengo listado de Eventos Populares en la ciudad de Los Angeles

In [49]:
fp = feedparser.parse('http://www.experiencela.com/rss/feeds/xlaevents.aspx?id=whatshot')

In [50]:
fechas = map(lambda e: fp.entries[e].datetime, range(len(fp.entries)))
resumenes = map(lambda e: fp.entries[e].title, range(len(fp.entries)))
#print fechas

In [51]:
from prettytable import PrettyTable
pt = PrettyTable()
pt.add_column('Fecha', fechas)
pt.add_column('Resumen', resumenes)
print pt

+-----------------------------------------+-------------------------------------------------+
|                  Fecha                  |                     Resumen                     |
+-----------------------------------------+-------------------------------------------------+
|  September 18, 2015; 6:00 PM - 7:00 PM  |    Sunset Ocean Swim Workout at Hermosa Beach   |
|       September 19, 2015; 2:30 PM       |              He-Man and She-Ra Day!             |
|  September 19, 2015; 2:00 PM - 4:30 PM  |  L.A. Conservancy's Modern Skyline Walking Tour |
| September 20, 2015; 10:30 AM - 12:00 PM |      Skirball Playdate: A Recipe for Family     |
|  September 23, 2015; 7:00 PM - 10:00 PM |              Rooftop Concert Series             |
|  September 24, 2015; 5:00 PM - 9:00 PM  |                    Beach Eats                   |
|  September 25, 2015; 6:00 PM - 8:00 PM  |         Robotic Cartography - Scale Up!         |
|  September 25, 2015; 6:00 PM - 7:00 PM  | Sunset Ocean Swi